# Week 3 project

Importing Essential Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium



    we shall be reading the following website: 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
    Then we would check for status code and headers



In [2]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695
200
{'Date': 'Fri, 27 Mar 2020 18:37:39 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Server': 'mw1266.eqiad.wmnet', 'X-Powered-By': 'PHP/7.2.26-1+0~20191218.33+debian9~1.gbpb5a340+wmf1', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Expires': 'Fri, 27 Mar 2020 18:37:39 GMT', 'Last-Modified': 'Wed, 18 Mar 2020 17:12:23 GMT', 'Backend-Timing': 'D=366891 t=1585334259536409', 'X-ATS-Timestamp': '1585334260', 'Content-Encoding': 'gzip', 'X-Varnish': '330360154', 'Age': '3', 'X-Cache': 'cp5010 miss, cp5008 pass', 'X-Cache-Status': 'pass', 'Server-Timing': 'cache;desc="pass"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=27-Mar-2020;Path=/;HttpOnly;secure;Expires=Tue, 28 Apr 2

Now we will proceed to create a dataframe and clean the data to gather the information that is relevant to us.

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])


In [4]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
df.head()
df.shape

(211, 3)

Here we will group the dataframe by postalcodes then we will keep the first borough and join the neighbourhoods

In [5]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

Now we will check out if it has worked correctly using 'M5R' as an example.

In [6]:
df[df['PostalCode']=='M5R']

,PostalCode,Borough,Neighborhood
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"


Now we shall convert the above data into a pandas dataframe

Now we will check the shape again

In [7]:
df.shape

(103, 3)

In [12]:






# Merge rows with the same Postcode
df_assigned = df.groupby(['PostalCode', 'Borough'], as_index=False).agg(lambda x: ', '.join(set(x)))

# Set 'Not assigned' Neigbourhood to be equal to Borough
na_nb_idx =df_assigned['Neighborhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighborhood'] = df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['PostalCode'] == 'M7A']



,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [13]:
df.shape

(103, 3)

In [18]:
# since we are unable to get coordinates using geocoder or wget, we will use the csv file directly after having downloaded it
df_cord= pd.read_csv('Geospatial_Coordinates.csv')
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df= pd.concat([df,df_cord], axis=1)
df.head()
df.drop('PostalCode', axis=1, inplace=True)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [23]:
df.head(20)

,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",M1K,43.727929,-79.262029
7,Scarborough,"Clairlea, Golden Mile, Oakridge",M1L,43.711112,-79.284577
8,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",M1M,43.716316,-79.239476
9,Scarborough,"Birch Cliff, Cliffside West",M1N,43.692657,-79.264848
